# 01_datawork

Munkhtsetseg

Library

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

# Import the dataset and remove the duplicates

Import the dataset from the directory of: ~/Data Input/Preprocessing data/Preprocessing data.csv, assign the dataset as object of df:

In [ ]:
df <- read.csv("~/Data Input/Preprocessing data/Preprocessing data.csv")

Remove the duplicates with the function of distinct(), assign the dataset as df_01:

In [ ]:
df_00 <- df |>
  distinct()

## Produce a table with missing data

In [ ]:
df_000 <- df_00 |>
   filter(Year<2019 & Year >2008)



df_01 <- df_000 |>
   select(Year, Month, Day, Hour, PM2, PM10, Visibility, WD, WS,   OPC, Station.name, Date) |>
  rowwise() |>
filter((Year != 2019 & Station.name == "Zamynuud") | 
       (Year < 2016 & Station.name == "Sainshand") |
       (Year < 2021 & Station.name == "UB") |
       (Year < 2017 & Station.name == "Dalanzadgad")
       )

For date options as year, month, etc:

In [ ]:
Table_NA_by_Year <- df_01 |>
  group_by(Station.name,Year) |> 
  summarise(NA_date = sum(!is.na(Date)),
            NA_PM2 = sum(is.na(PM2)),
            NA_PM10 = sum(is.na(PM10)),
            NA_Vis = sum(is.na(Visibility)),
            NA_WD = sum(is.na(WD)),
            NA_WS = sum(is.na(WS)),
            NA_OPC = sum(is.na(OPC))
            ) 

`summarise()` has grouped output by 'Station.name'. You can override using the
`.groups` argument.

# A tibble: 35 × 9
# Groups:   Station.name [4]
   Station.name  Year NA_date NA_PM2 NA_PM10 NA_Vis NA_WD NA_WS NA_OPC
   <chr>        <int>   <int>  <int>   <int>  <int> <int> <int>  <int>
 1 Dalanzadgad   2009    8760    715     929    659   748   748   8760
 2 Dalanzadgad   2010    8784    921    1086    756   787   787   8784
 3 Dalanzadgad   2011    8760   2652    3309   1759  2394  2394   8760
 4 Dalanzadgad   2012    5088   1074    3016    693  1412  1412   5088
 5 Dalanzadgad   2013    6096   1766    1809   2479  1240  1240   6096
 6 Dalanzadgad   2014    7800    843     921   6068  1482  1482   7800
 7 Dalanzadgad   2015    8760   1539    1587   8115  2635  2635   8760
 8 Dalanzadgad   2016    6288   1654    1613   5995  3306  3306   6288
 9 Sainshand     2009    8688    376     424    423   587   587   8688
10 Sainshand     2010    8784   2557    2577   1113  1210  1210   8784
# ℹ 25 more rows

For station

In [ ]:
Table_NA_by_Station <- df_01 |>
  group_by(Station.name) |> 
  summarise(NA_date = sum(!is.na(Date)),
            NA_PM2 = sum(is.na(PM2)),
            NA_PM10 = sum(is.na(PM10)),
            NA_Vis = sum(is.na(Visibility)),
            NA_WD = sum(is.na(WD)),
            NA_WS = sum(is.na(WS)),
            NA_OPC = sum(is.na(OPC))
  ) 
Table_NA_by_Station

# A tibble: 4 × 8
  Station.name NA_date NA_PM2 NA_PM10 NA_Vis NA_WD NA_WS NA_OPC
  <chr>          <int>  <int>   <int>  <int> <int> <int>  <int>
1 Dalanzadgad    60336  11164   14270  26524 14004 14004  60336
2 Sainshand      59040  11727   11929   9320  8527  8527  59040
3 UB             76656   7879    8716   3770  4053  4053  43415
4 Zamynuud       67392   8880   10075   3444  4960  4960  67392

By percentages

In [ ]:
Table_NA_by_Station_percentage <- df_01 |>
  group_by(Station.name, Year) |> 
  summarise(NA_date = sum(!is.na(Date)),
            NA_PM2 = sum(is.na(PM2)),
            NA_PM10 = sum(is.na(PM10)),
            NA_Vis = sum(is.na(Visibility)),
            NA_WD = sum(is.na(WD)),
            NA_WS = sum(is.na(WS)),
            NA_OPC = sum(is.na(OPC))
  ) |> 
  mutate(
            NA_PM2p = if_else(NA_date != NA_PM2, NA_PM2/NA_date*100, 0)
  ) |>
  do({
    PM10_perc <- .$NA_PM2*100/.$NA_date; 
      missing_PM10 <-mean(PM10_perc); 
      attr(missing_PM10, "data.all") <- PM10_perc; 
    PM2_perc <- .$NA_PM10*100/.$NA_date; 
      missing_PM2 <- mean(PM2_perc); 
      attr(missing_PM2, "data.all") <- PM2_perc;
    Vis_perc <- .$NA_Vis*100/.$NA_date; 
      missing_Vis <- mean(Vis_perc); 
      attr(missing_Vis, "data.all") <- Vis_perc; 
    WS_perc <- .$NA_WS*100/.$NA_date; 
      missing_WS <- mean(WS_perc); 
      attr(missing_WS, "data.all") <- WS_perc; 
    WD_perc <- .$NA_WD*100/.$NA_date; 
      missing_WD <- mean(WD_perc); 
      attr(missing_WD, "data.all") <- WD_perc; 
  data.frame(missing_PM2, missing_PM10, missing_Vis, missing_WS, missing_WD)}) 

`summarise()` has grouped output by 'Station.name'. You can override using the
`.groups` argument.

# A tibble: 4 × 6
# Groups:   Station.name [4]
  Station.name missing_PM2 missing_PM10 missing_Vis missing_WS missing_WD
  <chr>              <dbl>        <dbl>       <dbl>      <dbl>      <dbl>
1 Dalanzadgad         25.7         19.2       44.5       24.3       24.3 
2 Sainshand           20.0         19.7       15.7       14.6       14.6 
3 UB                  11.9         11.0        4.53       4.85       4.85
4 Zamynuud            14.4         12.7        5.49       7.44       7.44

# Note that:

We use the data in the period of 2009-2018, which has been regarded as a monitoring work stabilized since 2008 when is the beginning of the monitoring. According to NIES, site maintenance was consistent up to 2018. - Sainshand site, data 2009-2015 get used; - Dalanzad site: 2009-2016. - UB: 2009-2018 - Zamyn uud: 2009-2018

# Remove the spikes, and produce an extended table

Remove the spikes in the datasets, and produce the table with NA, with removed spikes; express it in a percentages. \### Remove the spikes Method 1. Mean value +- (3-5)SD - Find Monthly mean

In [ ]:
df_02 <- df_01 |>
  filter(Station.name=="UB" ) |>
  drop_na(PM2, PM10) |>
    dplyr::mutate_at(vars(PM2, PM10), 
                   ~ ifelse(near(., mean(.), sd(.) * 3), ., "Outlier")) |>
  dplyr::filter_if(is.character, any_vars(. == "Outlier"))
df_02

# A tibble: 1,798 × 12
    Year Month   Day  Hour PM2   PM10  Visibility    WD    WS   OPC Station.name
   <int> <int> <int> <int> <chr> <chr>      <int> <int> <dbl> <int> <chr>       
 1  2009     1     2    17 Outl… 0.29        3622   141 0.524    NA UB          
 2  2009     1     3    12 Outl… 0.446       2399   109 0.117    NA UB          
 3  2009     1     3    13 Outl… 0.288       1347    17 0.492    NA UB          
 4  2009     1     3    14 Outl… 0.504       1241    12 0.829    NA UB          
 5  2009     1     3    15 Outl… 0.478       1341    11 0.39     NA UB          
 6  2009     1     3    16 Outl… 0.449       2945   136 0.123    NA UB          
 7  2009     1     3    18 Outl… 0.341       1436    13 0.742    NA UB          
 8  2009     1     3    19 Outl… 0.397       1847    13 0.453    NA UB          
 9  2009     1     3    20 Outl… 0.297       3359    22 0.462    NA UB          
10  2009     1     4     2 Outl… 0.311       3167    96 0.759    NA UB          
# ℹ 1

In [ ]:
df_02 <- df_01 |>
  drop_na(PM2, PM10) |>
    dplyr::mutate_at(vars(PM2, PM10), 
                   ~ ifelse(near(., mean(., .by = c(Station.name, Month)), sd(.) * 3), ., "Outlier")) |>
  dplyr::filter_if(is.character, any_vars(. == "Outlier"))
df_02

# A tibble: 4,014 × 12
    Year Month   Day  Hour PM2   PM10  Visibility    WD    WS   OPC Station.name
   <int> <int> <int> <int> <chr> <chr>      <int> <int> <dbl> <int> <chr>       
 1  2009     1     3    15 Outl… 0.292       3444   119 0.856    NA Dalanzadgad 
 2  2009     1     5    13 Outl… 0.419       1383   260 1.7      NA Dalanzadgad 
 3  2009     1     5    14 Outl… 0.415       1072   266 1.84     NA Dalanzadgad 
 4  2009     1     5    15 Outl… 0.466       1099   261 0.83     NA Dalanzadgad 
 5  2009     1     5    16 Outl… 0.509       1814   260 0.788    NA Dalanzadgad 
 6  2009     1     6     0 Outl… 0.547        744   248 1.23     NA Dalanzadgad 
 7  2009     1     6     1 Outl… 0.728       1093   277 0.738    NA Dalanzadgad 
 8  2009     1     6     2 Outl… 0.597       1723     0 1.62     NA Dalanzadgad 
 9  2009     1     6     3 Outl… 0.33        8186    95 1.1      NA Dalanzadgad 
10  2009     1     6    11 Outl… 0.39        1150   258 1.48     NA Dalanzadgad 
# ℹ 4

## Save dataset in folder: 01_data_raw

# Tidy data

## Fill the missing data

Method 1. Fill the gap Method 2. Relationship equation Method 3. Look-up table

## Save dataset in folder: 02_data_tidy